Face web scraping do forecasts de DXY da pagina  https://www.forecasts.org/dollar-forecast.htm

gera df con os títulos ao consultar (symbol , url)

In [93]:
import sqlite3
import pandas as pd

# Conectar ao banco de dados SQLite
conn = sqlite3.connect("sqforecasts.db")

# Criar a consulta SQL
query = "SELECT symbol, url FROM tbtitulos"

# Criar o DataFrame com os dados da consulta
dftitulos = pd.read_sql(query, conn)

# Fechar a conexão
conn.close()

# Exibir o DataFrame
display(dftitulos)

,symbol,url
0,DXY,https://www.forecasts.org/dollar-forecast.htm
1,US10Y,https://www.forecasts.org/10yrT.htm
2,USIR,https://www.forecasts.org/inflation.htm
3,SPX,https://www.forecasts.org/stpoor.htm


In [95]:
url= str(dftitulos.iloc[0]['url'])
print(url)

https://www.forecasts.org/dollar-forecast.htm


In [97]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# URL da página de previsão do U.S. Dollar Index


# Função para obter os dados do índice
def get_fore_dados(url,symbol):
    # Fazer a requisição HTTP para a página
    response = requests.get(url)
    response.raise_for_status()  # Verifica se houve erro na requisição

    # Criar o objeto BeautifulSoup para processar o HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # Localizar a tabela pelo ID (caso tenha um identificador)
    table = soup.find("table", {"id": "forecast"})
    
    # Extrair todas as linhas da tabela
    rows = table.find_all("tr")[1:]  # Ignora o cabeçalho

    # Criar lista para armazenar os dados extraídos
    data_list = []
    
    for row in rows:
        cols = row.find_all("td")
        month = cols[1].text.strip()
        date = pd.to_datetime(month)  # Convertendo a string para uma data válida
        forecast_value = float(cols[2].text.strip())
        avg_error = float(cols[3].text.replace("±", "").strip())

        data_list.append([month, date, forecast_value, avg_error])

    # Criar DataFrame
    df_forecasts_scrap = pd.DataFrame(data_list, columns=["Month", "Date", "Forecast Value", "Avg Error"])

    # --- Capturar a data de atualização ---
    updated_text = soup.find("time").text.strip()
    updated_date = pd.to_datetime(updated_text.replace("Updated: ", ""))  # Converter para formato datetime

    return df_forecasts_scrap, updated_date, symbol


url = str(dftitulos.iloc[3]['url'])
symbol = str(dftitulos.iloc[3]['symbol'])

# Chamar a função para obter os dados
df_forecasts_scrap, last_updated, symbol = get_fore_dados(url,symbol)

# Exibir os dados
display(symbol)
display(f"Última atualização: {last_updated}")
display(df_forecasts_scrap)


'SPX'

'Última atualização: 2025-05-02 00:00:00'

,Month,Date,Forecast Value,Avg Error
0,Apr 2025,2025-04-01,5358.77,0.0
1,May 2025,2025-05-01,5539.00,52.0
2,Jun 2025,2025-06-01,5211.00,59.6
3,Jul 2025,2025-07-01,5290.00,64.1
4,Aug 2025,2025-08-01,5088.00,67.2
5,Sep 2025,2025-09-01,5248.00,69.7
6,Oct 2025,2025-10-01,5338.00,71.7
7,Nov 2025,2025-11-01,5473.00,73.4
8,Dec 2025,2025-12-01,5578.00,74.9


In [99]:
unicidade = symbol + str(last_updated)
print (unicidade)

SPX2025-05-02 00:00:00


In [103]:
import sqlite3

# Conectar ao banco de dados SQLite
conn = sqlite3.connect("sqforecasts.db")
cursor = conn.cursor()

# Tentar inserir os dados na tabela, capturando possíveis erros
try:
    query = "INSERT INTO tbforecasts (symbol, data, unicidade) VALUES (?, ?, ?)"
    cursor.execute(query, (symbol, str(last_updated), unicidade))
    conn.commit()
    print("Dados inseridos com sucesso na tabela tbforecasts!")
except sqlite3.IntegrityError:
    print("Esta atualização já existe no banco de dados.")

# Fechar a conexão
conn.close()

OperationalError: database is locked

In [91]:
conn.execute("PRAGMA journal_mode=WAL;")

OperationalError: cannot change into wal mode from within a transaction

In [15]:
import sqlite3
import pandas as pd

# Conectar ao banco de dados SQLite
conn = sqlite3.connect("sqforecasts.db")
cursor = conn.cursor()

# --- Passo 1: Inserir na tabela tbforecasts ---
cursor.execute("""
    INSERT INTO tbforecasts (symbol, data, descripzao) 
    VALUES (?, ?, ?)
""", ("DXY", str(last_updated), "Previsão do U.S. Dollar Index (DXY)"))

# Obter o ID gerado automaticamente
id_forecasts = cursor.lastrowid

print(f"Novo registro adicionado em tbforecasts com id_forecasts = {id_forecasts}")

# --- Passo 2: Inserir os dados do df_dxy na tabela tbforecastsdados ---
for _, row in df_dxy.iterrows():
    cursor.execute("""
        INSERT INTO tbforecastsdados (id_forecasts, data, value, error) 
        VALUES (?, ?, ?, ?)
    """, (id_forecasts, str(row["Date"]), row["Forecast Value"], row["Avg Error"]))

print(f"Registros adicionados na tabela tbforecastsdados para id_forecasts = {id_forecasts}")

# Confirmar as alterações e fechar conexão
conn.commit()
conn.close()

print("Dados exportados com sucesso para o banco de dados SQLite!")

Novo registro adicionado em tbforecasts com id_forecasts = 1
Registros adicionados na tabela tbforecastsdados para id_forecasts = 1
Dados exportados com sucesso para o banco de dados SQLite!


Bono 10 y

In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os

# URL da página de previsão dos Treasury Yields
url = "https://www.forecasts.org/10yrT.htm"

# Arquivo para armazenar a última atualização
UPDATE_FILE = "last_update.txt"

# Função para obter os dados da tabela
def get_treasury_data():
    # Fazer requisição HTTP para obter o conteúdo da página
    response = requests.get(url)
    response.raise_for_status()  # Verifica se houve erro na requisição

    # Processar o HTML com BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")

    # Localizar a tabela pelo ID "forecast"
    table = soup.find("table", {"id": "forecast"})
    rows = table.find_all("tr")[1:]  # Ignorar cabeçalho

    # Criar lista para armazenar os dados extraídos
    data_list = []

    for row in rows:
        cols = row.find_all("td")
        month = cols[1].text.strip()
        date = pd.to_datetime(month)  # Converter para formato datetime
        forecast_value = float(cols[2].text.strip())
        avg_error = float(cols[3].text.replace("±", "").strip())

        data_list.append([month, date, forecast_value, avg_error])

    # Criar DataFrame
    df_treasury = pd.DataFrame(data_list, columns=["Month", "Date", "Forecast Value", "Avg Error"])

    # Capturar a data de atualização
    updated_text = soup.find("time").text.strip()
    updated_date = pd.to_datetime(updated_text.replace("Updated: ", ""))  # Converter para formato datetime

    return df_treasury, updated_date

# Função para verificar e salvar atualização
def check_for_update():
    df_treasury, last_updated = get_treasury_data()

    # Verificar se há um arquivo de última atualização salvo
    if os.path.exists(UPDATE_FILE):
        with open(UPDATE_FILE, "r") as file:
            last_saved_date = pd.to_datetime(file.read().strip())

        # Se houver uma nova atualização, salvar novo CSV e atualizar o arquivo de controle
        if last_updated > last_saved_date:
            print(f"Nova atualização encontrada em {last_updated}! Salvando dados...")
            df_treasury.to_csv("treasury_forecast.csv", index=False)

            with open(UPDATE_FILE, "w") as file:
                file.write(str(last_updated))
        else:
            print("Nenhuma atualização nova encontrada.")
    else:
        # Primeira execução - salvar a data
        with open(UPDATE_FILE, "w") as file:
            file.write(str(last_updated))
        print("Primeira execução do script. Criando CSV inicial.")
        df_treasury.to_csv("treasury_forecast.csv", index=False)

# Executar a verificação automática
check_for_update()

Nenhuma atualização nova encontrada.
